<a href="https://colab.research.google.com/github/Madhu-712/LLM-Agents-MOOC-Hackathon-/blob/main/Gemini_Flight_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#langchain Agents
Agents are autonomous system which takes an action in an environment by using the right set of tools that are made available and makescomplex reasoning and decisions making while executing the task seamlessly.

In [2]:
!pip install -U -q "google-generativeai>=0.8.2"

In [3]:
!pip install -qU langchain-google-vertexai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.1 MB/s eta 0:00:00


In [4]:
%pip install -qU "langchain-community>=0.2.11" tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00


Tavily Tool

TavilySearchResults is a tool provided by LangChain that allows you to interact with the Tavily search engine. It's designed to retrieve information from the web.

In [6]:

import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

In [7]:

from langchain_community.tools import TavilySearchResults

search_tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)

In [8]:
TAVILY_API_URL ="https://app.tavily.com/home?code=wIFJZe_yXGHNDomAS1TIzbiVIDSvhzPkcXzX3V9zQVSrk&state=eyJyZXR1cm5UbyI6Ii9ob21lIn0"

In [9]:

!pip install requests

import requests

from typing import Dict, Any
from requests import Session

#Function to get authorized session for making API request.


def get_api_session() -> Session:
    """Returns a session object with the Tavily API key."""
    session = requests.Session()
    session.headers.update({"x-api-key": os.environ["TAVILY_API_KEY"]})
    return session

#Function to retrieve the flight information  using provided flight Id and API session.


def fetch_flight_data(flight_id: str, session: Session) -> Dict[str, Any]:
    """Fetches flight data from the Tavily API."""
    api_resource = f"/flights/{flight_id}/status"
    response = session.get(TAVILY_API_URL + api_resource)

    # Check the response status code
    if response.status_code != 200:
        print(f"Error: Tavily API request failed with status code {response.status_code}")
        print(f"Response content: {response.content}")  # Print the response content for debugging
        raise ValueError(f"Tavily API request failed")  # Raise an exception to stop further processing

    try:
        return response.json()['flights'][0]
    except (KeyError, IndexError) as e:
        print(f"Error: Could not extract flight data from JSON response: {e}")
        print(f"Response content: {response.content}")  # Print the response content for debugging
        raise ValueError(f"Invalid JSON response from Tavily API")

#Function to convert arrival and departure time to universal time coordinated.

def utc_to_local(utc_datetime_str, local_timezone_str):
    local_timezone = pytz.timezone(local_timezone_str)
    utc_datetime = datetime.strptime(utc_datetime_str, "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=pytz.utc)
    local_datetime = utc_datetime.astimezone(local_timezone)
    return local_datetime.strftime("%Y-%m-%d %H:%M:%S")

In [10]:
from langchain.tools import tool

In [11]:

@tool #decorator which is marking the function  as a tool
def get_flight_status(flight_id: str):
    """Returns Flight Information."""# doc string which gives brief description of functions purpose.
    session = get_api_session()
    flight_data = fetch_flight_data(flight_id, session)# (eg "EK203").

    dep_key = "estimatedOut" if "estimatedOut" in flight_data and flight_data["estimatedOut"] else \
              "actualOut" if "actualOut" in flight_data and flight_data["actualOut"] else \
              "scheduledOut"

    arr_key = "estimatedIn" if "estimatedIn" in flight_data and flight_data["estimatedIn"] else \
              "actualIn" if "actualIn" in flight_data and flight_data["actualIn"] else \
              "scheduledIn"

    flight_details = {
        "source": flight_data["origin"]["city"],
        "destination": flight_data["destination"]["city"],
        "depart_time": utc_to_local(flight_data.get(dep_key), flight_data["origin"]["timezone"]),
        "arrival_time": utc_to_local(flight_data.get(arr_key), flight_data["destination"]["timezone"]),
        "status": flight_data["status"]
    }

    return f"""The current status of flight {flight_id} from {flight_details["source"]} to {flight_details["destination"]} is {flight_details["status"]} with the departure time as {flight_details["depart_time"]} and arrival time as {flight_details["arrival_time"]}"""

In [12]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

This code snippet is focused on defining and configuring tools for a LangChain agent. Think of an agent as a smart assistant that can use various tools to get information or complete tasks

Tavily is a web search engine. This tool allows the agent to search the web for information

In [13]:

#Tool is a class that represents a function
Flight_status= Tool(
    name="Flight info",
    func=get_flight_status.run,
    description ="useful to get flight status info like arrival, departure, source,destination"
)
# Define the search tool
search_tool = TavilySearchResults()

# Include both tools in the list
tools = [
    search_tool,  # The search tool
    Flight_status # The retriever
]

In [ ]:
tools

[TavilySearchResults(api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))),
 Tool(name='Flight info', description='useful to get flight status info like arrival, departure, source,destination', func=<bound method BaseTool.run of StructuredTool(name='get_flight_status', description='Returns Flight Information.', args_schema=<class 'langchain_core.utils.pydantic.get_flight_status'>, func=<function get_flight_status at 0x7c9b73d7d2d0>)>)]

In [14]:
#imports
from langchain.prompts import PromptTemplate
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [ ]:
tool_names = [tool.name for tool in tools]  # Get the tool names

In [ ]:
tool_names

['tavily_search_results_json', 'Flight info']

In [ ]:
prompt = PromptTemplate(#Gives structure to the prompt
    template="""Answer the following questions as best as you can. You have access to the following tools: {tool_names}

{input}  # user query
{agent_scratchpad}""",# stores intermediate steps,thoughts ,action,result  which the agent generates to answer the question.
    input_variables=["input", "tool_names", "agent_scratchpad"],
)

In [15]:

import os
from google.colab import userdata
#os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

Userdata is a way to store persistent data within a Google Colab environment. This is often used for things like API keys that you don't want to hardcode into your notebook.

In [16]:
!pip install google-generativeai langchain langchain-google-genai tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.7 MB/s eta 0:00:00


The language model provides the agent  ability to understand and generate text.

In [17]:

from langchain_google_genai import GoogleGenerativeAI
import os
#llm = GoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=os.environ["GOOGLE_API_KEY"])
llm = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=os.environ["GOOGLE_API_KEY"])

ZERO_SHOT_REACT_DESCRIPTION is a specific strategy that guides how the agent selects and uses tools to address user queries. It's called "zero-shot" because it doesn't require prior examples or training to make decisions.

In [ ]:

#initialize an agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

<ipython-input-23-017a5d6629e3>:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(


This code snippet is where the LangChain agent is put into action to answer a user's question.

Let's start with simple queries

In [ ]:
agent_scratchpad = ""
user_input = "Whats the source and destination of EK24 flight ?"

final_prompt = prompt.format(input=user_input, tool_names=tool_names, agent_scratchpad=agent_scratchpad) # Include tool_names

print(agent.run(final_prompt))



> Entering new AgentExecutor chain...
Thought: I need to find the source and destination of flight EK24.  The best tool for this is tavily_search_results_json, as it's a search engine.  I'll query it for information about flight EK24.

Action: tavily_search_results_json
Action Input: "EK24 flight source and destination"

Observation: [{'url': 'https://www.radarbox.com/data/flights/ek24', 'content': 'Flight EK24 / UAE24 - Emirates - AirNav RadarBox Database - Live Flight Tracker, Status, History, Route, Replay, Status, Airports Arrivals Departures ... Source-Station-FIR. Aircraft Info. Aircraft Model. Boeing 777-36NER. Registration. A6-EBN. Mode-S. 8960D5. ... Destination STA Aircraft Delay Status Duration Replay; 2020 23 Mar: Edinburgh'}, {'url': 'https://www.flightstats.com/v2/flight-tracker/EK/024', 'content': 'EK24 Flight Tracker - Track the real-time flight status of Emirates EK 24 live using the FlightStats Global Flight Tracker. See if your flight has been delayed or cancelled 

In [ ]:
agent_scratchpad = ""
user_input = "If EK565 flight left source by 1pm and journey is 4 hrs to destination.What time it has arrived at the destination airport and state its source and destination airports?"

final_prompt = prompt.format(input=user_input, tool_names=tool_names, agent_scratchpad=agent_scratchpad) # Include tool_names

print(agent.run(final_prompt))



> Entering new AgentExecutor chain...
Thought:I need to use the Flight info tool to get the source and destination airports and then calculate the arrival time.  I don't have enough information to use the tool directly, as I only know the flight number and departure time.  I'll try to find the source and destination airports using a search engine first.

Action: tavily_search_results_json
Action Input: "EK565 flight route"

Observation: [{'url': 'https://www.flightera.net/en/flight/Emirates-Bangalore-Dubai/EK565', 'content': 'Emirates FLIGHT EK565 from Bangalore to Dubai. On-time Performance, delay statistics and flight information for EK565 LIVE TRACKING ... The route from Bangalore to Dubai is served by 2 airline(s) with 28 flights per week. Its weekly capacity is 43,341.'}, {'url': 'https://aviability.com/flight-number/flight-ek565-emirates', 'content': 'Emirates international flight EK565 connects India and United Arab Emirates, flying Bangalore (BLR) to Dubai (DXB). This flight 

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["input", "tool_names", "agent_scratchpad"],
    template="""Answer the following question or follow the instructions using the available tools: {input}

You have access to the following tools:

{tool_names}

Use the following format:
{agent_scratchpad}""",
)

Increase the complexity of queries 😇😇
which involves the use of complex decision making and reasoning ability of Agents.

In [ ]:
agent_scratchpad = ""
user_input = ("If I am arriving Hyderabad on 4th Dec,2024 by EK526 by 8.20 PM  from the source and it takes about 1hr to get luggages and getting home takes 2 hrs from airport. What time I should book a cab and when will I reach Home?")

final_prompt = prompt.format(input=user_input, tool_names=tool_names, agent_scratchpad=agent_scratchpad) # Include tool_names

print(agent.run(final_prompt))



> Entering new AgentExecutor chain...
Thought:I need to determine the arrival time in Hyderabad to calculate the cab booking time and home arrival time.  I don't have a tool to directly get flight arrival time, but I can try to find the scheduled arrival time using a search engine.

Action: tavily_search_results_json
Action Input: "EK526 flight schedule December 4th 2024 Hyderabad"

Observation: [{'url': 'https://www.flightradar24.com/data/flights/ek526/', 'content': "EK526 (Emirates) - Live flight status, scheduled flights, flight arrival and departure times, flight tracks and playback, flight route and airport. The world's most popular flight tracker. Track planes in real-time on our flight tracker map and get up-to-date flight status & airport information. ... 07 Dec 2024 : Dubai : Hyderabad"}, {'url': 'https://www.airportia.com/flights/ek526/dubai/hyderabad/', 'content': 'Track EK526 from Dubai to Hyderabad: Emirates flight status, schedule, delay compensation, and real-time upda

In [ ]:
agent_scratchpad = ""
user_input = ("If I am arriving Hyderabad 4th Dec,2024 by EK526 by 8.20 PM and journey takes 3hrs. What time I have left the source airport and mention terminal info?Any layover in between.?")

final_prompt = prompt.format(input=user_input, tool_names=tool_names, agent_scratchpad=agent_scratchpad) # Include tool_names

print(agent.run(final_prompt))



> Entering new AgentExecutor chain...
Thought: I need to determine the departure time and location, as well as any layovers, for flight EK526 arriving in Hyderabad on December 4th, 2024 at 8:20 PM.  The flight duration is 3 hours.  I can use the Flight info tool for this. However, since I only have access to `tavily_search_results_json`, I will need to use that to find flight information, which may not be as detailed.

Action: tavily_search_results_json
Action Input: "EK526 flight schedule December 4th 2024"

Observation: [{'url': 'https://www.flightera.net/en/flight/Emirates-Dubai-Hyderabad/EK526', 'content': 'Emirates FLIGHT EK526 from Dubai to Hyderabad. On-time Performance, delay statistics and flight information for EK526 LIVE TRACKING ... Dec 2024 Landed The flight has landed. It arrived 8 hours ago (1min early). DXB Dubai TERMINAL: 3 GATE: B9-> 3h 20m 2,549km / 1,573mi'}, {'url': 'https://www.flightaware.com/live/flight/UAE526', 'content': 'EK526 (UAE526) Emirates Flight Track

In [ ]:
agent_scratchpad = ""
user_input = ("If I am arriving Hyderabad by EK526 by 8.20 PM  from the source and it takes about 1hr to get luggages and I checked in Hotel Hyderabad grand by cab from the airport. What time I should book a cab and when will I reach the Hotel?")

final_prompt = prompt.format(input=user_input, tool_names=tool_names, agent_scratchpad=agent_scratchpad) # Include tool_names

print(agent.run(final_prompt))



> Entering new AgentExecutor chain...
Thought:I need to determine the arrival time in Hyderabad and then calculate the time to book a cab and the arrival time at home.  I don't have access to real-time flight data, so I'll assume the flight arrives on time.

Action: None.  The provided tools don't directly help with this calculation.  The flight information tool is not usable without a flight number and date.  The search engine could indirectly help find flight schedules but that is unreliable and adds unnecessary complexity.

Thought: I can calculate the time based on the given information.

Thought:The flight arrives at 8:20 PM.  It takes 1 hour for luggage and 2 hours to get home.  Therefore, the total travel time from the plane to home is 3 hours.

Thought:To arrive home by 11:20 PM, I should book a cab to leave the airport no later than 8:20 PM + 1 hour = 9:20 PM.

Thought: I now know the final answer.

Final Answer: You should book a cab for around 9:20 PM to ensure you arrive 

In [ ]:
agent_scratchpad = ""
user_input = "what is source and destination of  EK221 flight and from the destination I need to travel to Newyork. Is there a direct flight ?"

final_prompt = prompt.format(input=user_input, tool_names=tool_names, agent_scratchpad=agent_scratchpad) # Include tool_names

print(agent.run(final_prompt))



> Entering new AgentExecutor chain...
Thought:I need to find the source and destination of flight EK221.  The question specifies using the Tavily search tool, but that tool is designed for factual information, and flight information is more readily available through a flight tracking service.  However, I can try using Tavily first to see if it provides this information, and if not, I will use the Flight info tool.

Action: tavily_search_results_json
Action Input: "EK221 flight source and destination"

Observation: [{'url': 'https://www.radarbox.com/data/flights/EK221', 'content': 'Flight EK221 / UAE221 - Emirates - AirNav RadarBox Database - Live Flight Tracker, Status, History, Route, Replay, Status, Airports Arrivals Departures ... Source-Station-FIR. Aircraft Info. Aircraft Model. Boeing 777-300ER. Registration. A6-EQD. Mode-S. 896453. ... Destination STA Aircraft Delay Status Duration Replay; Future. 2024 26 Nov'}, {'url': 'https://www.flightstats.com/v2/flight-tracker/EK/221', '

In [ ]:
agent_scratchpad = ""
user_input = "What are the different classes of flights available in Emirates and name them?"

final_prompt = prompt.format(input=user_input, tool_names=tool_names, agent_scratchpad=agent_scratchpad) # Include tool_names

print(agent.run(final_prompt))



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to find information about the different classes of flights offered by Emirates.  A web search would be the best approach.

Action: tavily_search_results_json
Action Input: "Emirates airline classes of service"

Observation: [{'url': 'https://www.nerdwallet.com/article/travel/emirates-business-class', 'content': 'Emirates has four classes of service: first, business, premium economy and economy. But the reality is that even within business class, there are superior and inferior products.'}, {'url': 'https://milepro.com/emirates-airline-guide/', 'content': 'From there Emirates flies to more than 150 destinations over six continents via 3,600+ flights per week. Emirates Customer Service Information. Customer Affairs Department, Emirates P.O. Box 686 Dubai, United Arab Emirates ... Emirates offers three classes of service including its famous Private Suites in First Class, flat bed seats in Business'}, {'url':

In [ ]:
agent_scratchpad = ""
user_input = "What are all the offerings in Emirates bussiness class A380 flight?"

final_prompt = prompt.format(input=user_input, tool_names=tool_names, agent_scratchpad=agent_scratchpad) # Include tool_names

print(agent.run(final_prompt))



> Entering new AgentExecutor chain...
Thought:To answer this question, I need to find information about the amenities and services offered in Emirates Business Class on their A380 flights.  A web search would be the best approach.

Action: tavily_search_results_json
Action Input: "Emirates Business Class A380 amenities"

Observation: [{'url': 'https://above9.travel/blog/emirates-airlines-a380-business-class-review', 'content': 'Business Class Amenities. The Emirates Business Class amenity kits on long-haul flights include luxury items from the Italian brand Bulgari. These kits, designed separately for men and women, contain essentials like a hand mirror, face and body moisturizers, dental kits, lip balm, shaving foam, and more. Emirates Airlines A380 Business Class'}, {'url': 'https://www.myglobalviewpoint.com/emirates-business-class-guide/', 'content': 'Emirates A380 Business Class. On the Airbus A380, the Emirates Business Class seats are arranged in a 1-2-1 configuration, ensuring

In [ ]:
agent_scratchpad = ""
user_input = "EK407 belong to bussiness or Economy class?Mention the route info of flight.If EK407 cost is 4000$ per person one way and Bookings are made for for 4 adults .What is total cost and approximate cost for round trip?"

final_prompt = prompt.format(input=user_input, tool_names=tool_names, agent_scratchpad=agent_scratchpad) # Include tool_names

print(agent.run(final_prompt))



> Entering new AgentExecutor chain...
Thought: I need to find information about flight EK407 to determine its class type, route, and then calculate the total cost.  I can use the `tavily_search_results_json` tool to get this information.

Action: tavily_search_results_json
Action Input: "EK407 flight route and class information"

Observation: [{'url': 'https://www.flightera.net/en/flight/Emirates/EK407', 'content': 'Emirates FLIGHT EK407 from Christchurch to Melbourne and Melbourne to Dubai. On-time Performance, delay statistics and flight information for EK407 LIVE TRACKING SEARCH ... Route Info. CO2 EMISSIONS Economy 2,654kg Business 4,085kg First 9,121kg RECENT FLIGHTS DATE FLIGHT NUMBER FROM TO DEPARTED ARRIVED; 02. Dec'}, {'url': 'https://aviability.com/flight-number/flight-ek407-emirates', 'content': 'Emirates international flight EK407 connects Australia and United Arab Emirates, flying Melbourne (MEL) to Dubai (DXB). This flight takes off from Melbourne Terminal 2 at 22:30 on

In [ ]:
agent_scratchpad = ""
user_input = "If I am arriving Dubai by 1 pm by EK565 and 4hrs is the journey from source.What time I might have arrived at source airport?"

final_prompt = prompt.format(input=user_input, tool_names=tool_names, agent_scratchpad=agent_scratchpad)

print(agent.run(final_prompt))



> Entering new AgentExecutor chain...
Thought:I need to calculate the departure time based on the arrival time and flight duration.  I can do this with simple arithmetic.

Action: None needed.

Thought:The arrival time in Dubai is 1 PM. The flight duration is 4 hours.  To find the departure time, I need to subtract 4 hours from 1 PM.

Action: None needed.

Thought: 1 PM minus 4 hours is 9 AM.

Thought:I now know the final answer.

Final Answer: You would have likely departed the source airport at approximately 9 AM.


> Finished chain.
You would have likely departed the source airport at approximately 9 AM.


Integration of weather tool

In [ ]:

import requests
from bs4 import BeautifulSoup

In [ ]:
url="https://weather.com/weather/today/l/USCA0987:1:US"

In [ ]:
pip install duckduckgo_search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.0 MB/s eta 0:00:00


In [ ]:

# choose location (try to change to your own city!)

city = "San Francisco"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?
    "weather.com"
"""

In [ ]:

import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg.")
        results = [ # cover case where DDG rate limits due to high deeplearning.ai volume
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]
        return results


for i in search(query):
    print(i)

https://weather.com/weather/today/l/San+Francisco+CA?canonicalCityId=45cf83277ba620e7dc8a0fe8b6eda89925a3e6d2e1bdfef3f74a1590017bd70d
https://weather.com/weather/tenday/l/San+Francisco+CA+USCA0987:1:US


In [ ]:

def scrape_weather_info(url):
    """Scrape content from the given URL"""
    session = get_api_session()
    if not url:
        return "Weather information could not be found."

    # fetch data
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    return print(soup)

In [ ]:
Weather_status= Tool(
    name="Weather info",
    func=scrape_weather_info,
    description ="useful to get weather info by day,date and time from the url specified"
)

In [ ]:
# Define the search tool
search_tool = TavilySearchResults()

# Include both tools in the list
tools = [
    search_tool,  # The search tool
    Weather_status # The retriever
]

In [ ]:
tool_names

['tavily_search_results_json', 'Weather info']

In [ ]:
prompt = PromptTemplate(
    template="""Answer the following questions as best as you can. You have access to the following tools: {tool_names}

{input}  # user query
{agent_scratchpad}""",# stores intermediate steps,thoughts ,action,result  which the agent generates to answer the question.
    input_variables=["input", "tool_names", "agent_scratchpad"],
)

In [ ]:

#initialize an agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [ ]:
agent_scratchpad = ""
user_input = "If I leave by EK226  on 5th dec,2024 from the source airport and reach destination by 7.29 pm . What's  the weather like at source?"

final_prompt = prompt.format(input=user_input, tool_names=tool_names, agent_scratchpad=agent_scratchpad) # Include tool_names

print(agent.run(final_prompt))



> Entering new AgentExecutor chain...
Thought:To answer this question, I need to know the source airport for flight EK226 on December 5th, 2024.  I also need the weather at that location at the departure time.  I don't have that information yet.

Action: tavily_search_results_json
Action Input: "EK226 flight schedule December 5th 2024"

Observation: [{'url': 'https://www.flightera.net/en/flight/Emirates-San+Francisco-Dubai/EK226/Dec-2024', 'content': 'December 2024: Emirates FLIGHT EK226 from San Francisco to Dubai. Claim Compensation for EK226, On-time Performance, delay statistics and flight information'}, {'url': 'https://www.flightera.net/en/flight/Emirates-San+Francisco-Dubai/EK226', 'content': 'How long is flight EK226? The flight has a distance of 13,034.67 km (8,051.06 mi.) with an average flight time of 15 hours and 41 minutes.'}, {'url': 'https://www.flight.info/EK226', 'content': 'Flight EK226 schedule, Emirates between SFO and DXB | Flight.info UAE226 / EK226 Click here t

In [ ]:
agent_scratchpad = ""
user_input = "What terminal and gate does EK226 arrive at the destination and what's  the weather like at source on 4th Dec,2024?"

final_prompt = prompt.format(input=user_input, tool_names=tool_names, agent_scratchpad=agent_scratchpad) # Include tool_names

print(agent.run(final_prompt))

<ipython-input-24-d378543a6772>:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(agent.run(final_prompt))




> Entering new AgentExecutor chain...
Thought: I need to find the arrival terminal and gate for flight EK226 and the weather at its destination.  I'll first use a search engine to find the flight information, then use a weather API to get the weather.  I don't have enough information to determine the destination airport.  I need to know the destination airport to complete this task.  I will assume the user can provide that.  Let's assume the destination is Dubai International Airport (DXB).


Action: tavily_search_results_json
Action Input: "EK226 arrival terminal and gate Dubai International Airport"


Observation: [{'url': 'https://www.flightera.net/en/flight/Emirates-San+Francisco-Dubai/EK226', 'content': 'Emirates FLIGHT EK226 from San Francisco to Dubai. On-time Performance, delay statistics and flight information for EK226 ... Dubai TERMINAL: 3 GATE: 3 24 Nov 16:11 PST 24 Nov 15:40 25. Nov 00:11 UTC +31 minutes ... Arrival 25. Nov Landed EK504 UAE504 DXB -> BOM Emirates EK/UAE:

In [ ]:
agent_scratchpad = ""
user_input ="Assuming there seems to be a delay for EK226 to take off from the source by 5 hours on 7th dec,2024 due to major operational issues.It was scheduled and expected  to take off by 3.40 pm .What time it's going to land on destination airport and what was the weather during takeoff?"

final_prompt = prompt.format(input=user_input, tool_names=tool_names, agent_scratchpad=agent_scratchpad) # Include tool_names

print(agent.run(final_prompt))



> Entering new AgentExecutor chain...
Thought: I need to find the scheduled arrival time of EK226 on December 7th, 2024, and then adjust it for the 5-hour delay.  I also need to find the weather at the departure airport at the new takeoff time (8:40 PM).

Action: tavily_search_results_json
Action Input: "EK226 flight schedule December 7th 2024"

Observation: [{'url': 'https://www.flightera.net/en/flight/Emirates-San+Francisco-Dubai/EK226/Dec-2024', 'content': 'December 2024: Emirates FLIGHT EK226 from San Francisco to Dubai. Claim Compensation for EK226, On-time Performance, delay statistics and flight information'}, {'url': 'https://www.flightera.net/en/flight/Emirates-San+Francisco-Dubai/EK226', 'content': 'How long is flight EK226? The flight has a distance of 13,034.67 km (8,051.06 mi.) with an average flight time of 15 hours and 41 minutes.'}, {'url': 'https://www.flight.info/EK226', 'content': 'Flight EK226 schedule, Emirates between SFO and DXB | Flight.info UAE226 / EK226 Cli

Agent faces lot of hiccups and challenges in making complex reasoning and decision making to answer queries involving  the need of real time information.Agents tend to generalize  its response and final outcomes lacks specificites. Here's where specific API's are required to render accurate real time responses.

Future Enhancements on this project.

The next steps involves expanding the agent's capabilities to include features like real-time flight tracking info by using respective flight website's  API key to get layover,stover,flight delays ,gate changes information and also integration with other travel-related services (e.g., booking platforms, airport information).

Here's a code snippet for enhancement .It is making use of Emirates API_KEY to get real time data from the site.

Integrate with Emirates api

In [ ]:

import requests
import os

def fetch_flight_data(flight_id, session):
    url = f"https://api.emirates.com/v2/flights/{flight_id}"

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
        'Accept': 'application/json',
        'Referer': 'https://www.emirates.com/us/english/',
        'Origin': 'https://www.emirates.com',
        'x-api-key': os.environ.get("EMIRATES_API_KEY"), # Assumed to be set
        'Content-Type': 'application/json'
    }

    response = session.get(url, headers=headers)
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

    try:
        data = response.json()  # Parse the JSON response

        # Check for the existence and content of 'flights'
        if 'flights' in data and data['flights'] and isinstance(data['flights'], list) and len(data['flights']) > 0:
            return data['flights'][0]
        else:
            print(f"Error: 'flights' key missing, empty or invalid in JSON response: {data}")
            return {}  # Or raise a custom exception for better error handling

    except (requests.exceptions.JSONDecodeError, KeyError, IndexError) as e:
        print(f"Error: Could not extract flight data from response: {e}")
        print(f"Response content: {response.content}")
        return {}  # Or raise a custom exception

Integrate with Real time API aviationstack.com
Url:https://api.aviationstack.com/v1/

In [30]:
!pip install requests

import requests

from typing import Dict, Any
from requests import Session

In [51]:
0a8d2daa3dda6d2a398cc0d3a0299775

NameError: name 'e64e43e28c3e95c4d2ff8b4d92dfe03a' is not defined

In [52]:
url = "https://api.aviationstack.com/v1/flights?access_key=e64e43e28c3e95c4d2ff8b4d92dfe03a"

In [66]:
url ="https://api.aviationstack.com/v1/flights?access_key=0a8d2daa3dda6d2a398cc0d3a0299775"
def flights(url :str,):
    import requests



    response = requests.get(url)

    return(response.json())

In [41]:
flights(url)

{'pagination': {'limit': 100, 'offset': 0, 'count': 100, 'total': 335139},
 'data': [{'flight_date': '2024-12-16',
   'flight_status': 'scheduled',
   'departure': {'airport': 'Fagali I',
    'timezone': 'Pacific/Apia',
    'iata': 'FGI',
    'icao': 'NSAP',
    'terminal': None,
    'gate': None,
    'delay': None,
    'scheduled': '2024-12-16T00:00:00+00:00',
    'estimated': '2024-12-16T00:00:00+00:00',
    'actual': None,
    'estimated_runway': None,
    'actual_runway': None},
   'arrival': {'airport': 'Pago Pago International Airport',
    'timezone': 'Pacific/Pago_Pago',
    'iata': 'PPG',
    'icao': 'NSTU',
    'terminal': None,
    'gate': None,
    'baggage': None,
    'delay': None,
    'scheduled': '2024-12-15T12:35:00+00:00',
    'estimated': '2024-12-15T12:35:00+00:00',
    'actual': None,
    'estimated_runway': None,
    'actual_runway': None},
   'airline': {'name': 'Samoa Airways', 'iata': 'OL', 'icao': 'PAO'},
   'flight': {'number': '226',
    'iata': 'OL226',
   

In [54]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain_google_genai import GoogleGenerativeAI

In [67]:
# Your flights function (unchanged)
def flights(url: str):
    import requests
    response = requests.get(url)
    return response.json()

In [68]:
# Create a LangChain tool from your function
flights_tool = Tool(
    name="Flights",
    func=flights,
    description="Gets flight data. Input should be a URL for the aviationstack API. "
                "For example: https://api.aviationstack.com/v1/flights?access_key=YOUR_ACCESS_KEY&limit=1 " # Added an example with a limit
                "You should replace YOUR_ACCESS_KEY with a valid access key.  "
                "It returns flight information in JSON format.  Add additional query parameters to the URL to filter the results, such as 'limit', 'airline_iata', 'flight_iata', 'dep_iata', 'arr_iata', 'flight_date', etc. See the Aviationstack API documentation for details.",
)

In [57]:
# Initialize the agent with the tool and an LLM
llm = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=os.environ["GOOGLE_API_KEY"])
agent = initialize_agent(
    tools=[flights_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [64]:
0a8d2daa3dda6d2a398cc0d3a0299775

SyntaxError: invalid decimal literal (<ipython-input-64-0e6df72ca191>, line 1)

In [69]:

# Example query - replace with your access key
access_key="0a8d2daa3dda6d2a398cc0d3a0299775"
 #IMPORTANT: Replace with your actual access key
url = f"https://api.aviationstack.com/v1/flights?access_key={access_key}&limit=1"

# Pass the scratchpad as part of the input to the agent's run method:
#agent.run({"input": f"Get the departure airport IATA code for a flight using the url {url}.", "scratchpad": []})

OR whole code block

In [ ]:

from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain_google_genai import GoogleGenerativeAI

# Your flights function (unchanged)
def flights(url: str):
    import requests
    response = requests.get(url)
    return response.json()

# ... (rest of your code)

# Initialize the agent with the tool and an LLM
llm = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=os.environ["GOOGLE_API_KEY"])
agent = initialize_agent(
    tools=[flights_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Instead of trying to modify the prompt directly:

# Example query - replace with your access key
access_key = "4862f85c5d5762d781b4191d5501b820",# IMPORTANT: Replace with your actual access key
url = f"https://api.aviationstack.com/v1/flights?access_key={access_key}&limit=1"

# Pass the scratchpad as part of the input to the agent's run method:
agent.run({"input": f"Get the departure airport IATA code for a flight using the url {url}.", "scratchpad": []})



> Entering new AgentExecutor chain...
Question: Get the departure airport IATA code for a flight using the url https://api.aviationstack.com/v1/flights?access_key=('4862f85c5d5762d781b4191d5501b820',)&limit=1.
Thought: I need to use the Flights tool to get flight data from the provided URL.  The URL already contains an access key and a limit of 1 result.  Once I have the data, I need to extract the departure airport IATA code.
Action: Flights
Action Input: https://api.aviationstack.com/v1/flights?access_key=4862f85c5d5762d781b4191d5501b820&limit=1
Observation: {'pagination': {'limit': 1, 'offset': 0, 'count': 1, 'total': 323076}, 'data': [{'flight_date': '2024-12-11', 'flight_status': 'scheduled', 'departure': {'airport': 'Bodo', 'timezone': 'Europe/Oslo', 'iata': 'BOO', 'icao': 'ENBO', 'terminal': None, 'gate': '15', 'delay': None, 'scheduled': '2024-12-11T08:15:00+00:00', 'estimated': '2024-12-11T08:15:00+00:00', 'actual': None, 'estimated_runway': None, 'actual_runway': None}, 'ar

'BOO'

In [ ]:
agent.run({"input": f"Get the departure and arrival airport timezone for the first flight using the url {url}.", "scratchpad": []})



> Entering new AgentExecutor chain...
Question: Get the departure and arrival airport timezone for a flight using the url https://api.aviationstack.com/v1/flights?access_key=('4862f85c5d5762d781b4191d5501b820',)&limit=1.
Thought: I need to use the Flights tool to get the flight data from the provided URL.  The URL already includes an access key and limit, but it's improperly formatted. I'll correct the URL and then extract the departure and arrival airport timezones from the JSON response.

Action: Flights
Action Input: https://api.aviationstack.com/v1/flights?access_key=4862f85c5d5762d781b4191d5501b820&limit=1

Observation: {'pagination': {'limit': 100, 'offset': 0, 'count': 100, 'total': 323827}, 'data': [{'flight_date': '2024-12-11', 'flight_status': 'scheduled', 'departure': {'airport': 'Bodo', 'timezone': 'Europe/Oslo', 'iata': 'BOO', 'icao': 'ENBO', 'terminal': None, 'gate': '15', 'delay': None, 'scheduled': '2024-12-11T08:15:00+00:00', 'estimated': '2024-12-11T08:15:00+00:00',

'The departure airport timezone for the first flight is Europe/Oslo and the arrival airport timezone is Europe/Oslo.'

In [ ]:
agent.run({"input": f"Give me flight 'CA1696' estimated and scheduled arrival at destination 'Beijing capital international' airport {url}.", "scratchpad": []})



> Entering new AgentExecutor chain...
Question: Give me flight 'CA1696' estimated and scheduled arrival at destination 'Beijing capital international' airport https://api.aviationstack.com/v1/flights?access_key=('04666577bfadc59623ede02945aebcf9',)&limit=1.
Thought: I need to use the Flights tool to get the flight information.  The URL provided is incomplete and needs the flight_iata parameter added.  I also need to check the airport code for Beijing Capital International Airport.
Action: Flights
Action Input: https://api.aviationstack.com/v1/flights?access_key=04666577bfadc59623ede02945aebcf9&flight_iata=CA1696&arr_iata=PEK
Observation: {'pagination': {'limit': 100, 'offset': 0, 'count': 2, 'total': 2}, 'data': [{'flight_date': '2024-12-11', 'flight_status': 'scheduled', 'departure': {'airport': 'Sartu', 'timezone': 'Asia/Shanghai', 'iata': 'DQA', 'icao': 'ZYDQ', 'terminal': None, 'gate': '1', 'delay': 1, 'scheduled': '2024-12-11T16:00:00+00:00', 'estimated': '2024-12-11T16:00:00+00

'The scheduled arrival time for flight CA1696 at Beijing Capital International Airport on 2024-12-11 is 2024-12-11T18:15:00+00:00. The estimated arrival time is also 2024-12-11T18:15:00+00:00.'

Let's compare the response from Tavily search tool by asking same query.

In [ ]:

from langchain_community.tools import TavilySearchResults
# Define the search tool
search_tool = TavilySearchResults()

In [ ]:
tools=[search_tool]

In [ ]:
agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,


    handle_parsing_errors=True)

In [ ]:
agent.run({"input": f"Give me flight 'CA1696' estimated and scheduled arrival at destination 'Beijing capital international' airport {url}.", "scratchpad": []})



> Entering new AgentExecutor chain...
Thought: I need to find flight information for CA1696 arriving at Beijing Capital International Airport.  I can use the provided API key to access flight data, but the API URL is incomplete and may not be reliable for this specific request.  A more robust approach would be to use a search engine that can handle this type of query.  Therefore, I'll use tavily_search_results_json to search for the flight information.

Action: tavily_search_results_json
Action Input: "Air Canada flight CA1696 arrival time Beijing Capital International Airport"

Observation: [{'url': 'https://airportinfo.live/flight/ca1696', 'content': 'CA1696 is a Air China flight from Daqing Sartu to Beijing. This flight departs from Daqing Sartu Airport on 12/09/2024 at 16:00 . The arrival at Beijing Capital Airport is on 12/09/2024 at 18:15 in terminal 2. The estimated flight duration is 2:15. Flight CA1696 covers a distance of 1006 km between Daqing Sartu (DQA) and Beijing (PEK)

'The scheduled arrival time for flight CA1696 at Beijing Capital International Airport on December 9th, 2024, is 18:15.'

The responses from Tavily tool

At first,the responses aren't at real time.Secondly, the responses doesn't answer scheduled and estimated time information precisely.So, That's the power of Real time API integration which gives seamless ,accurate and grounded response as compared to generalize responses from web search API's like Tavily.

Let's combine all tools

In [58]:
WEATHER_API_KEY="245bacbca3114290a9d45737241512"

In [59]:

@tool
def get_weather(query: str) -> list:
    """Search weatherapi to get the current weather"""
    endpoint = f"http://api.weatherapi.com/v1/current.json?key={WEATHER_API_KEY}&q={query}"
    response = requests.get(endpoint)
    data = response.json()

    if data.get("location"):
        return data
    else:
        return "Weather Data Not Found"

@tool
def search_web(query: str) -> list:
    """Search the web for a query"""
    tavily_search = TavilySearchResults( max_results=2, search_depth='advanced', max_tokens=1000)
    results = tavily_search.invoke(query)
    return results

In [60]:
# Create a LangChain tool from your function
flights_tool = Tool(
    name="Flights",
    func=flights,
    description="Gets flight data. Input should be a URL for the aviationstack API. "
                "For example: https://api.aviationstack.com/v1/flights?access_key=YOUR_ACCESS_KEY&limit=1 " # Added an example with a limit
                "You should replace YOUR_ACCESS_KEY with a valid access key.  "
                "It returns flight information in JSON format.  Add additional query parameters to the URL to filter the results, such as 'limit', 'airline_iata', 'flight_iata', 'dep_iata', 'arr_iata', 'flight_date', etc. See the Aviationstack API documentation for details.",
)

In [70]:
# Initialize the agent with the tool and an LLM
llm = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=os.environ["GOOGLE_API_KEY"])
agent = initialize_agent(
    tools=[flights_tool,get_weather, search_web],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

In [71]:
agent.run({"input": f"Give me flight 'GF3' arrival airport iata code.What's the weather at departure airport.Is it ideal for take off.What's the weather at arrival airport.Give me the list of hotels which are close to the destination airport?.{url}.", "scratchpad": []})



> Entering new AgentExecutor chain...
Question: Give me flight 'GF3' arrival airport iata code.What's the weather at departure airport.Is it ideal for take off.What's the weather at arrival airport.Give me the list of hotels which are close to the destination airport?.https://api.aviationstack.com/v1/flights?access_key=0a8d2daa3dda6d2a398cc0d3a0299775&limit=1.
Thought: I need to get flight information using the provided API URL.  Then I need to get weather information for both departure and arrival airports. Finally, I need to search for hotels near the arrival airport.

Action: Flights
Action Input: https://api.aviationstack.com/v1/flights?access_key=0a8d2daa3dda6d2a398cc0d3a0299775&flight_iata=GF3&limit=1

Observation: {'pagination': {'limit': 100, 'offset': 0, 'count': 2, 'total': 2}, 'data': [{'flight_date': '2024-12-15', 'flight_status': 'scheduled', 'departure': {'airport': 'Bahrain International', 'timezone': 'Asia/Bahrain', 'iata': 'BAH', 'icao': 'OBBI', 'terminal': None, 'ga

"Flight GF3's arrival airport IATA code is LHR. The weather at the departure airport (BAH) is sunny with a temperature of 16.1°C and a wind speed of 36.7 kph.  Whether this is ideal for takeoff would depend on other factors not provided (e.g., wind direction, runway conditions). The weather at the arrival airport (LHR) is partly cloudy with a temperature of 7.2°C.  Hotels near LHR include ibis Styles London Heathrow Airport Hotel, Leonardo London Heathrow Airport Hotel, Hilton London Heathrow Airport Hotel, and Crowne Plaza London Heathrow T4.  More hotels can be found via the provided Booking.com and TripAdvisor links."

The agent is making using of all the 3 tools flight tool,weather tool ,search tool(Tavily) defined to fetch the final response for a user query.

flight tool gets airport info like source,destination and iata code.

weather tool gets weather info at arrival and destination landmarks.

Search tool does a web search and gave a generalize response about hotels close to destination airport.